# Genetic Improvement Demo

### Create the initial population of Genomes

In [1]:
# system imports
import os, sys
from tqdm import tqdm 


# set the project root directory (~/projects/GIMC)
project_root = os.path.expanduser("~/projects/GIMC")

# add project root to sys.path
sys.path.append(project_root)

# project imports
from genetic_improvement.ollamachat import OllamaChat
from genetic_improvement.config import USER_PROMPT, UNIT_TEST_CODE, SYSTEM_PROMPT, NUM_VARIANTS, BSI_CLASSIFICATION
from genetic_improvement.genome import Genome

# CONSTS
POPULATION_SIZE = 8


In [2]:
# get the initial population from the LLM
population = []
for i in tqdm(range(POPULATION_SIZE // NUM_VARIANTS)):
    ollama_chat = OllamaChat(model="qwen3-coder:latest", system_prompt=SYSTEM_PROMPT, temperature=0.3)
    chat_response = ollama_chat.chat(user_text=USER_PROMPT, stream=False)

    # parse the chat response and add it to the population
    variants = OllamaChat.parse_variants(chat_response)
    candidates = OllamaChat.submit_variants(variants, classification=BSI_CLASSIFICATION)
    for candidate in candidates:
        if candidate is not None:
            genome = Genome(candidate_hash=candidate)
            population.append(genome)

 50%|█████     | 1/2 [00:27<00:27, 27.19s/it]

Code submitted successfully:  0e513f6346c71fc2107aabb2282e6a017215e96543bcd104dab0af3a6a7b7168
Code submitted successfully:  04eda47900c69e845e94f7743bf9b26c56f5bf973ad6fe8c40737b476951e16e
Code submitted successfully:  2158f1984903a26ca6e2a64554c2d8ba03979fcaf8246ad523f06e9433bc9751
Code submitted successfully:  cce0a337663d46053dbef305df84af28af0e28e7ac63cc0b0f276b3933120e9b


100%|██████████| 2/2 [01:06<00:00, 33.06s/it]

Code submitted successfully:  2002ab1a6878e19730d7c54f597276b25a00e84d010a98b95aac0e585d5dab6a
Code submitted successfully:  edefd969b839235dc0cee838db5fc06768947f9f624036b5a70667b28ad25c6a
Code submitted successfully:  901f1786ad89954ee741f4a331184b137375e1ab801f81b68d0f6ab211676a9d
Code submitted successfully:  196f552dd821e39680765539f85e20418e52b3496da475db0ca613f9db827198


In [3]:
# get the status of each candidate in the population
for genome in population:
    print(f"Candidate {genome.candidate_hash}")


Candidate 0e513f6346c71fc2107aabb2282e6a017215e96543bcd104dab0af3a6a7b7168
Candidate 04eda47900c69e845e94f7743bf9b26c56f5bf973ad6fe8c40737b476951e16e
Candidate 2158f1984903a26ca6e2a64554c2d8ba03979fcaf8246ad523f06e9433bc9751
Candidate cce0a337663d46053dbef305df84af28af0e28e7ac63cc0b0f276b3933120e9b
Candidate 2002ab1a6878e19730d7c54f597276b25a00e84d010a98b95aac0e585d5dab6a
Candidate edefd969b839235dc0cee838db5fc06768947f9f624036b5a70667b28ad25c6a
Candidate 901f1786ad89954ee741f4a331184b137375e1ab801f81b68d0f6ab211676a9d
Candidate 196f552dd821e39680765539f85e20418e52b3496da475db0ca613f9db827198


In [4]:
# Database setup
import json

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Sample, Tag, Analysis, Candidate

with open(os.path.join(project_root, "settings.json")) as f:
    settings = json.load(f)
DATABASE_URL = settings['sqlalchemy_database_uri']
engine = create_engine(DATABASE_URL, echo=False)
Session = sessionmaker(bind=engine)

In [9]:
for genome in population:
    print(genome.get_candidate())

Candidate ID: 0e513f6346c71fc2107aabb2282e6a017215e96543bcd104dab0af3a6a7b7168, Classification: com, Status: 3, Fitness: 8.933e-02, F1: 2.500e-01, F2: 3.333e-01, F3: 8.109e-05, Analysis ID: 17947, Error: None
Candidate ID: 04eda47900c69e845e94f7743bf9b26c56f5bf973ad6fe8c40737b476951e16e, Classification: com, Status: 3, Fitness: 2.252e-01, F1: 1.000e+00, F2: 3.333e-01, F3: 8.332e-02, Analysis ID: 17948, Error: None
Candidate ID: 2158f1984903a26ca6e2a64554c2d8ba03979fcaf8246ad523f06e9433bc9751, Classification: com, Status: 3, Fitness: 3.968e-03, F1: 3.704e-02, F2: 0.000e+00, F3: 0.000e+00, Analysis ID: None, Error: Compilation failed: 2 errors, 20 warnings
Candidate ID: cce0a337663d46053dbef305df84af28af0e28e7ac63cc0b0f276b3933120e9b, Classification: com, Status: 3, Fitness: 4.870e-03, F1: 4.545e-02, F2: 0.000e+00, F3: 0.000e+00, Analysis ID: None, Error: Compilation failed: 7 errors, 0 warnings
Candidate ID: 2002ab1a6878e19730d7c54f597276b25a00e84d010a98b95aac0e585d5dab6a, Classificatio